In [1]:
from prompter.interpreter.parsing import Parser

parser = Parser()

In [2]:
sample_text = """

##INFO_START
User name is John and wants to see if you can generate images.
##INFO_END

##REASONNING_START
Je dois créer une image pour montre mes capacités
##REASONNING_END

Oui je peux générer des images:
##ACTION_START id=img001
A cat at sunset, 1:1 ratio
##ACTION_END


##RESULT_START id=img001 name="NOT_HERE"
Image created at `images/cat.png`
##RESULT_END

J'ai crée une image de chat dans votre répertoire `images/cat.png`


##TEST_START name="NOT_HERE" is_here=False
Ceci est un test
##TEST_END

"""


blocks = parser.parse(sample_text)
for block in blocks:
    print(f"\nType: {block.type}")
    print(f"Params: {block.params}")
    print(f"Content:\n{block.content}")
    print("-" * 50)


Type: INFO
Params: {}
Content:
User name is John and wants to see if you can generate images.
--------------------------------------------------

Type: REASONNING
Params: {}
Content:
Je dois créer une image pour montre mes capacités
--------------------------------------------------

Type: TEXT
Params: {}
Content:
Oui je peux générer des images:
--------------------------------------------------

Type: ACTION
Params: {'id': 'img001'}
Content:
A cat at sunset, 1:1 ratio
--------------------------------------------------

Type: RESULT
Params: {'id': 'img001', 'name': '"NOT_HERE"'}
Content:
Image created at `images/cat.png`
--------------------------------------------------

Type: TEXT
Params: {}
Content:
J'ai crée une image de chat dans votre répertoire `images/cat.png`
--------------------------------------------------

Type: TEST
Params: {'name': '"NOT_HERE"', 'is_here': 'False'}
Content:
Ceci est un test
--------------------------------------------------


---

In [3]:
from prompter.interpreter.helper import BlockHelper


In [4]:
print(
    BlockHelper.create_block(
        block_type="EXAMPLE",
        content="This is an example block.",
        params={
            "is_a_test": True
        }
    )
)

##EXAMPLE_START is_a_test=True
This is an example block.
##EXAMPLE_END


---

In [5]:
from typing import Dict
from prompter.interpreter.evaluator import Evaluator, ActionRegistry, ActionHandler


# Création d'un bloc avec le helper
action_block = BlockHelper.create_action("Generate image", "img001")
print("Created block:", action_block)

# Configuration du registre
registry = ActionRegistry()

class ImageHandler(ActionHandler):
    # Exemple de fonction de traitement
    def handle(self, content: str, params: Dict[str, str]) -> str:
        return f"Generated image from: {content}"

registry.register("default", ImageHandler())

# Création de l'evaluator
evaluator = Evaluator(registry)

# Parse et évalue les blocs
parser = Parser()
blocks = parser.parse(action_block)
results = evaluator.evaluate_blocks(blocks)

print("Results:", results)

Created block: ##ACTION_START id=img001
Generate image
##ACTION_END
Results: ['##RESULT_START id=img001\nGenerated image from: Generate image\n##RESULT_END']


In [6]:
evaluator.registry

ActionRegistry(handlers=[default: <ImageHandler()>])

In [7]:
evaluator.context

ActionContext(used_ids=[img001], results=[img001: Generated image from: Generate image])

---

In [8]:
sample_text = """

##ACTION_START type=goto id=redirect001
Ixabel, the user wants to start a project named 'python_compiler'. Can you assist with setting up the project directory and initial files?
##ACTION_END

##ACTION_START id=redirect002
{
    "action": "goto",
    "prompt": "Ixabel, the user wants to start a project named 'python_compiler'. Can you assist with setting up the project directory and initial files?"
}
##ACTION_END

"""

In [9]:
import json



In [10]:
# Configuration du registre
registry = ActionRegistry()

# Exemple de traitement
class DefaultHandler(ActionHandler):
    # Exemple de fonction de traitement
    def handle(self, content: str, params: Dict[str, str]) -> str:
        try:
            return json.loads(content)
        except:
            return f"[ERROR JSON] Default from: {content}"
    
class ImageHandler(ActionHandler):
    # Exemple de fonction de traitement
    def handle(self, content: str, params: Dict[str, str]) -> str:
        return f"Generated image from: {content}"

class GotoHandler(ActionHandler):
    # Exemple de fonction de traitement
    def handle(self, content: str, params: Dict[str, str]) -> str:
        return f"Go to Agent from: {content}"

registry.register("default", DefaultHandler())
registry.register("image", ImageHandler())
registry.register("goto", GotoHandler())

registry

ActionRegistry(handlers=[default: <DefaultHandler()>, image: <ImageHandler()>, goto: <GotoHandler()>])

In [11]:
# Création de l'evaluator
evaluator = Evaluator(registry)

# Parse et évalue les blocs
parser = Parser()
blocks = parser.parse(sample_text)
results = evaluator.evaluate_blocks(blocks)
for block, result in zip(blocks, results):
    print(block)
    print(result)
    print()

Block(type='ACTION', content="Ixabel, the user wants to start a project named 'python_compiler'. Can you assist with setting up the project directory and initial files?", params={'type': 'goto', 'id': 'redirect001'}, start_pos=2, end_pos=193)
##RESULT_START id=redirect001
Go to Agent from: Ixabel, the user wants to start a project named 'python_compiler'. Can you assist with setting up the project directory and initial files?
##RESULT_END

Block(type='ACTION', content='{\n    "action": "goto",\n    "prompt": "Ixabel, the user wants to start a project named \'python_compiler\'. Can you assist with setting up the project directory and initial files?"\n}', params={'id': 'redirect002'}, start_pos=195, end_pos=418)
##RESULT_START id=redirect002
{'action': 'goto', 'prompt': "Ixabel, the user wants to start a project named 'python_compiler'. Can you assist with setting up the project directory and initial files?"}
##RESULT_END



---


In [12]:
from prompter.interpreter.interpreter import Interpreter

In [13]:
# Création de l'interpréteur avec le handler par défaut
interpreter = Interpreter()


test_block = BlockHelper.create_action("Test action", "test001")
test_block_result = BlockHelper.create_result("Test result", "test001")

custom_block = BlockHelper.create_block(
    "ACTION",
    "Some custom action",
    {"id": "custom001", "type": "custom"}
)

sample_text = f"""
Bonjour !
{test_block}
"""


# Interprétation
results = interpreter.interpret(sample_text)
print(results)  # Affichera le warning du default_handler

interpreter.clear_context()



['##RESULT_START id=test001\nWARNING: No default handler registered for action with content: Test action\n##RESULT_END']


In [14]:
sample_text = f"""
Bonjour !
{test_block}

{test_block_result}

Here the custom block
{custom_block}
"""
print(sample_text)

# Ajout d'un handler personnalisé
class CustomHandler(ActionHandler):
    def handle(self, content: str, params: Dict[str, str]) -> str:
        return f"Custom handling of: {content}"

interpreter.register_handler("custom", CustomHandler())


# Nouvelle interprétation
results = interpreter.interpret(sample_text)
# Affichera le résultat du custom_handler
print("-" * 20)
for res in results:
    print(res, end='\n\n')


Bonjour !
##ACTION_START id=test001
Test action
##ACTION_END

##RESULT_START id=test001
Test result
##RESULT_END

Here the custom block
##ACTION_START id=custom001 type=custom
Some custom action
##ACTION_END

--------------------
##RESULT_START id=custom001
Custom handling of: Some custom action
##RESULT_END



---
---

In [15]:
from prompter.interpreter import Interpreter, ActionRegistry, ActionHandler, BlockHelper

sample_text = """
Bonjour !

##ACTION_START type=does_not_exist id=test0001
This type does not exist
##ACTION_END

##ACTION_START type=call id=img0001
Generate an image of a cat
##ACTION_END

##ACTION_START type=call id=img0001
Ignored due to same id
##ACTION_END

##ACTION_START type=help id=help0001
I need help with ...
##ACTION_END

"""

interpreter = Interpreter(registry=ActionRegistry())

class CallHandler(ActionHandler):
    def handle(self, content: str, params: Dict[str, str]) -> str:
        return f"Call handling of: {content}"
    
class HelpHandler(ActionHandler):
    def handle(self, content: str, params: Dict[str, str]) -> str:
        raise RuntimeError("Some error during HELP Action handling.")
    
interpreter.register_handler("call", CallHandler())
interpreter.register_handler("help", HelpHandler())

results = interpreter.interpret(sample_text)
for result in results:
    print(result, end="\n" * 2)

##ERROR_START
Error executing action: No handler registered for action type 'does_not_exist'
> Make sure to use an available `type` parameter.
##ERROR_END

##RESULT_START id=img0001
Call handling of: Generate an image of a cat
##RESULT_END

##RESULT_START id=help0001
Error executing action: Some error during HELP Action handling.
##RESULT_END

